In [1]:
knitr::opts_chunk$set(echo = TRUE)



# Spatial integration

The following functions were used to create a spatial integration dataset.


In [2]:
#libraries
library(sf)
library(tidyverse)
library(feather)

Linking to GEOS 3.9.1, GDAL 3.2.3, PROJ 7.2.1

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.5     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [3]:
spatialIndex = function(){
  # read the shapefiles, crs = 2193 NZGD200/NZTM200 
  # -> WARNING pretty big files <-
  #map.mesh = st_read("spatialData/statsnzmeshblock-2021-generalised-SHP/meshblock-2021-generalised.shp")
  map.sa2 = st_read("spatialData/statsnzstatistical-area-2-2020-generalised-SHP/statistical-area-2-2020-generalised.shp")
  map.fenz = st_read("spatialData/lds-fire-and-emergency-nz-localities-SHP/fire-and-emergency-nz-localities.shp")
  map.lds = st_read("spatialData/lds-nz-land-districts-SHP/nz-land-districts.shp")
  
  #intersect
  #index1 = st_intersection(map.mesh, map.sa2) %>% st_drop_geometry() #left out at this stage
  urbanIndex = st_nearest_feature(map.sa2, map.fenz) #get closest FENZ locality to SA2 -> urban
  ruralIndex = st_nearest_feature(map.fenz, map.sa2) #get closest SA2 to FENZ locality ->rural
  
  #filter urban and rural areas
  urbanSpatialIndex = cbind(map.sa2 %>% st_drop_geometry(), map.fenz[urbanIndex,] %>% st_drop_geometry()) %>% 
    filter(type == "SUBURB") # urban part
  ruralSpatialIndex = cbind(map.sa2[ruralIndex,] %>% st_drop_geometry(), map.fenz %>% st_drop_geometry()) %>% 
    filter(type != "SUBURB") # rural part
  
  #add region by intersection
  regionIndex = st_intersection(map.fenz, map.lds) %>% st_drop_geometry()
  
  #join urban and rural
  fullSpatialIndex = rbind(urbanSpatialIndex, ruralSpatialIndex)
  fullSpatialIndex = left_join(fullSpatialIndex, regionIndex)
  
  #write
  write_feather(fullSpatialIndex, "outputData/spatialIndexFull.feather")
}


In [4]:
cleanUp.export.SpatialIndex = function(){
  index = read_feather("outputData/spatialIndexFull.feather")
  index = index %>% 
  select(
    name, id.1,
    city_name, city_id,
    suburb_4th, id,
    SA22020__2, SA22020_V1
  ) %>% 
  rename(
    RegionName = name, regionID = id.1,
    TownCity = city_name, cityID = city_id,
    SuburbName = suburb_4th, SuburbID = id,
    SA2Name = SA22020__2, SA2ID = SA22020_V1
  ) 
  
  #write
  write_feather(index, "outputData/spatialIndexTrimmed.feather")
  write.csv(index, "outputData/spatialIndexTrimmed.csv")
  print(index)
}


In [5]:
main = function(){
  spatialIndex() #this will take some time
  cleanUp.export.SpatialIndex()
}
main()


Reading layer `statistical-area-2-2020-generalised' from data source 
  `/Users/ll/Documents/MADS/DATA422/AOJCCYDPLL_Data422GroupProject/spatialData/statsnzstatistical-area-2-2020-generalised-SHP/statistical-area-2-2020-generalised.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 2255 features and 6 fields (with 16 geometries empty)
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 1067061 ymin: 4701317 xmax: 2523320 ymax: 6242140
Projected CRS: NZGD2000 / New Zealand Transverse Mercator 2000
Reading layer `fire-and-emergency-nz-localities' from data source 
  `/Users/ll/Documents/MADS/DATA422/AOJCCYDPLL_Data422GroupProject/spatialData/lds-fire-and-emergency-nz-localities-SHP/fire-and-emergency-nz-localities.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 7375 features and 15 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 1089971 ymin: 4747979 xmax: 2470566 ymax: 6223164
Projected CRS: NZGD2000 / New Z

Warning message:
“attribute variables are assumed to be spatially constant throughout all geometries”
Joining, by = c("id", "parent_id", "suburb_4th", "suburb_3rd", "suburb_2nd", "suburb_1st", "type_order", "type", "city_id", "city_name", "has_addres", "start_date", "end_date", "majorlocal", "majorloc_1")



# A tibble: 7,912 × 8
   RegionName     regionID TownCity  cityID SuburbName  SuburbID SA2Name   SA2ID
   <chr>             <int> <chr>      <int> <chr>          <int> <chr>     <chr>
 1 North Auckland     1001 Kaitaia   100041 Ahipara         2793 Rangaunu… 1002…
 2 North Auckland     1001 Kaitaia   100041 Ahipara         2793 Rangitihi 1009…
 3 North Auckland     1001 Taipa     100099 Taipa           2806 Oruru-Pa… 1010…
 4 North Auckland     1001 Moerewa   104074 Moerewa         2287 Pakaraka  1036…
 5 North Auckland     1001 Hikurangi 100165 Glenbervie…     8666 Kauri     1053…
 6 North Auckland     1001 Whangarei 100137 Otangarei       1153 Kamo Cen… 1060…
 7 North Auckland     1001 Whangarei 100137 Onerahi         2838 Abbey Ca… 1067…
 8 North Auckland     1001 Whangarei 100137 Whau Valley     2251 Mairtown  1068…
 9 North Auckland     1001 Whangarei 100137 Raumanga        2843 Woodhill… 1070…
10 North Auckland     1001 Whangarei 100137 Port Whang…     2845 Tarewa    1074…
# … wi